# YouTube Transcript Generator
[![Open in Collab](https://img.shields.io/badge/Open_in_Collab-555?style=for-the-badge&logo=googlecolab&labelColor=gray&color=purple)](https://colab.research.google.com/github/therohitdas/Youtube-Transcript-Generator/blob/main/main.ipynb)
![GitHub License](https://img.shields.io/github/license/therohitdas/Youtube-Transcript-Generator?style=for-the-badge&color=blue) ![GitHub Repo stars](https://img.shields.io/github/stars/therohitdas/Youtube-Transcript-Generator?style=for-the-badge&logo=github)

## Overview 🌐

The YouTube Transcript Generator is a powerful tool designed to streamline the process of extracting and processing transcripts from YouTube videos. Whether you're looking to transcribe lectures, interviews, or any other video content, this project provides a convenient solution.

### How It Can Help 🚀

This tool is particularly useful for:
- **Note Taking:** Quickly convert YouTube videos into text format for easy note-taking.
- **Content Analysis:** Analyze and derive insights from video content by converting it into text data.
- **Chat Bot Training:** Use the generated transcripts to train chatbots, such as ChatGPT, for natural language understanding.
- **Archiving:** Create a textual archive of valuable information from YouTube videos. This can be particularly useful for archiving interviews, tutorials, or any content you'd like to reference later without the need to re-watch the video.
- **Personal Knowledge Base:** Build a personal knowledge base by extracting and processing transcripts from YouTube videos. This can aid in consolidating information on diverse topics in a readable and accessible format.
- **Accessibility Improvement:** Enhance accessibility for individuals who prefer or require text-based content. The tool can be used to generate transcripts with added punctuation, improving the overall readability of the content.

## Features 🛠️

- **Transcription:** Obtain raw transcripts from YouTube videos.
- **Punctuation:** Enhance transcripts by adding punctuation using [deep multilingual punctuation models](https://huggingface.co/oliverguhr/fullstop-punctuation-multilang-large).
- **Chapter Detection:** Identify and separate chapters in the video based on provided timestamps.
- **User-friendly:** Easy-to-use script with customizable parameters.

## Environment Variables 🌐

- `GOOGLE_API_KEY`: Set up your Google API key for video information retrieval. You will need to create a Project in the google cloud for this and enable the YouTube v3 API. This is optional, if you don't add it, the chapters will not be added.

## Script Parameters 📜
```python
url = 'https://www.youtube.com/watch?v=YOUR_VIDEO_ID' # youtu.be link works too
language = 'en'
punctuated = True # Default False, takes significantly more time when enabled on CPU, use T4 GPU type in google collab.
output_dir = '.' # add /content/drive/MyDrive/ to save content in You Google Drive
filename = "" # Leave empty for default filename: Video Title or Video Id
punctuation_model = '' # More info down below
verbose = True # To get logs
```
`language` use the language code to get the video. By default this module always picks manually created transcripts over automatically created ones, if a transcript in the requested language is available both manually created and generated.

`punctuation_model` values can be found at https://huggingface.co/oliverguhr/fullstop-punctuation-multilang-large#languages

## Support 💬

For any issues or feature requests, please [create an issue](https://github.com/therohitdas/Youtube-Transcript-Generator/issues).

## Acknowledgments 🙌

This script utilizes the [youtube-transcript-api](https://github.com/jdepoix/youtube-transcript-api) and [fullstop-punctuation-multilang-large](https://huggingface.co/oliverguhr/fullstop-punctuation-multilang-large) libraries. Special thanks to their contributors.

Feel free to adapt and use the script based on your requirements. Enjoy the convenience of YouTube transcript processing!

## Connect with me 📧
The best way to connect is to email me [namaste@theRohitDas.com](mailto:namaste@therohitdas.com)
- [x/therohitdas](https://x.com/therohitdas)
- [GitHub/therohitdas](https://github.com/therohitdas)

🚀 Happy transcribing!

In [ ]:
!pip install youtube-transcript-api deepmultilingualpunctuation nltk tqdm pip install google-api-python-client google-auth-oauthlib

In [ ]:
# Run this if you want to mount and store generated files in google drive.
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
import os
import logging
import re
import math
import youtube_transcript_api
from deepmultilingualpunctuation import PunctuationModel
import nltk
import warnings
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google.colab import userdata

In [ ]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

logging.basicConfig(level=logging.INFO, force=True)

In [ ]:
def clean_for_filename(title):
    # Define a regular expression to keep only alphanumeric characters, spaces, dots, hyphens, and various parentheses
    cleaned_title = re.sub(r'[^\w\s\.\-\(\)\[\]]', '', title)

    # Remove leading and trailing spaces
    return cleaned_title.strip()

def remove_music_tags(text):
    # Remove [Music] or [music]
    updated_text = re.sub(r'\[music\]', '', text, flags=re.IGNORECASE)
    return updated_text

def remove_period_after_hashes(text):
    # Remove . after # or ##, considering newline characters
    return re.sub(r'(#\.|##\.)', lambda match: match.group(1)[:-1], text)

def remove_escape_sequences(text):
    # Some old videos contain escape sequences like \n in their subtitle
    # Remove \n, \r\n, \t, \b, \r
    return re.sub(r'\\[nrtb]|\\r\n', '', text)

def remove_double_greater_than(text):
    # Replace occurrences of ">>" with an empty string
    cleaned_text = re.sub(r'>>', '', text)
    return cleaned_text

def add_punctuation(text, punctuation_model):
    if punctuation_model != "":
        model = PunctuationModel(model=punctuation_model)
    else:
        model = PunctuationModel()
        punctuated_text = model.restore_punctuation(text)
    return punctuated_text

def capitalize_sentences(sentences):
    # Capitalize the first letter of each sentence in a batch
    capitalized_sentences = [sentence[0].upper() + sentence[1:] for sentence in sentences]
    return capitalized_sentences

def parse_youtube_url(url):
    video_id_match = re.search(r'(?:youtube\.com\/.*?[?&]v=|youtu\.be\/)([^"&?\/\s]{11})', url)
    if video_id_match:
        return video_id_match.group(1)
    else:
        raise ValueError('Invalid YouTube URL')

def parse_chapters(description):
    lines = description.split("\n")
    regex = re.compile(r"(\d{0,2}:?\d{1,2}:\d{2})")
    chapters = []

    for line in lines:
        matches = regex.findall(line)
        if matches:
            ts = matches[0]
            title = line.replace(ts, "").strip()

            # Check if the title contains another timestamp and remove it
            title = re.sub(r'\d{0,2}:?\d{1,2}:\d{2}', '', title).strip().strip('-').strip().strip('-').strip()

            chapters.append({
                "timestamp": ts,
                "title": title,
            })

    return chapters

def get_transcript(video_id, language, video_info, verbose=True):
    transcript_list = youtube_transcript_api.YouTubeTranscriptApi.get_transcript(video_id, languages=[language])

    if video_info["title"] != "":
        transcript = f'# {video_info["title"]}\n\n'

    current_chapter_index = 0
    chapters = video_info["chapters"]
    logging.info(f"Transcript_List Length: {len(transcript_list)}, Chapter Length: {len(chapters)}")

    for i, line in enumerate(transcript_list):
        start_time = int(math.floor(line['start']))  # Floor and convert to integer

        # Check if current_chapter_index is within the valid range
        if 0 <= current_chapter_index < len(chapters):
            chapter_time = chapters[current_chapter_index]['timestamp']

            try:
                # Extract start time from the chapter timestamp
                chapter_start = chapter_time.strip()
                chapter_start_seconds = sum(int(x) * 60 ** i for i, x in enumerate(reversed(chapter_start.split(':'))))
                chapters[current_chapter_index]["title"] = chapters[current_chapter_index]["title"].strip()
                buffer_time = 2

                if start_time >= chapter_start_seconds - buffer_time:
                    logging.info(f'\n\n## {chapters[current_chapter_index]["title"]}\n')
                    current_chapter_index += 1
            except Exception as e:
                logging.error(f"Error processing chapter timestamp: {chapter_time}")
                logging.error(f"Error details: {e}")

        line['text'] = remove_music_tags(line['text'])
        line['text'] = remove_escape_sequences(line['text'])
        line['text'] = remove_double_greater_than(line['text'])
        if line['text']:
          transcript += line['text'].strip() + ' '

        # Log progress information
        if verbose and i % 100 == 0:  # Adjust the log frequency as needed
            logging.info(f"Processed {i} lines out of {len(transcript_list)}")

    return transcript

def process_and_save_transcript(video_id, video_info, language, generate_punctuated, output_dir, filename, verbose, punctuation_model):
    try:
        raw_transcript = get_transcript(video_id, language, video_info, verbose)
        logging.info("Raw Transcript Length: %d", len(raw_transcript))

        if generate_punctuated:
            with_punctuation = add_punctuation(raw_transcript, punctuation_model)
            with_punctuation = remove_period_after_hashes(with_punctuation)
            logging.info("Punctuation Char Length: %d", len(with_punctuation))
            sentences = nltk.sent_tokenize(with_punctuation)
            logging.info("Sentences to process, (punctuated): %d", len(sentences))
        else:
            sentences = nltk.sent_tokenize(raw_transcript)
            logging.info("Sentences to process, (raw): %d", len(sentences))

        # Capitalize sentences without batching
        capitalized_sentences = capitalize_sentences(sentences)

        double_linesep = os.linesep + os.linesep
        capitalized_transcript = double_linesep.join(capitalized_sentences)
        output_path = os.path.join(output_dir, f'{filename}.md')

        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(capitalized_transcript)

        if generate_punctuated:
            logging.info(f'Punctuated transcript saved to {output_path}')
        else:
            logging.info(f'Raw transcript saved to {output_path}')

    except Exception as e:
        logging.error(f'Error: {e}')

def getVideoInfo (video_id):
  try:
    # Set up Google API credentials using API key
    api_key =  userdata.get('GOOGLE_API_KEY') # Replace with your actual API key
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)
    request = youtube.videos().list(part="id,snippet",
                                id = video_id
        )
    response = request.execute()
    title = response['items'][0]['snippet']['title']
    description = response['items'][0]['snippet']['description']
    data = {"title" : title, "chapters" : parse_chapters(description)}
    return data
  except Exception as e:
    logging.error(f'Error: {e}')
    return {"title": "", "chapters": []}

## Example Usage:
```python
url = 'https://www.youtube.com/watch?v=YOUR_VIDEO_ID' # youtu.be link works too
language = 'en'
punctuated = True # Default False, takes significantly more time when enabled on CPU, use T4 GPU type in google collab.
output_dir = '.' # add /content/drive/MyDrive/ to save content in You Google Drive
filename = "" # Leave empty for default filename: Video Title or Video Id
punctuation_model = '' # More info down below
verbose = True # To get logs
```
`language` use the language code to get the video. By default this module always picks manually created transcripts over automatically created ones, if a transcript in the requested language is available both manually created and generated.

`punctuation_model` values can be found at https://huggingface.co/oliverguhr/fullstop-punctuation-multilang-large#languages

In [ ]:
url = 'https://www.youtube.com/watch?v=CBYhVcO4WgI'
language = 'en'
punctuated = True # Default False, takes significantly more time when enabled on CPU, use T4 GPU type in google collab.
output_dir = '.' # add /content/drive/MyDrive/ to save content in You Google Drive
filename = "" # Leave empty for default filename: Video Title or Video Id
punctuation_model = ''
verbose = True

In [ ]:
video_id = parse_youtube_url(url)
video_info = getVideoInfo(video_id)
filename = filename = filename or clean_for_filename(video_info["title"]) or clean_for_filename(video_id)

In [ ]:
process_and_save_transcript(video_id, video_info, language, punctuated, output_dir, filename, verbose, punctuation_model)

In [ ]:
# Download the Generated File
from google.colab import files
files.download(os.path.join(output_dir, f'{filename}.md'))